In [3]:
# --- Celda 1 Modificada ---
import pandas as pd
import geopandas as gpd
import os
from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic 
from langchain.agents import create_sql_agent
from langchain_community.utilities import SQLDatabase

print("Librerías para la Fase 3 (OpenAI y Anthropic) importadas.")

Librerías para la Fase 3 (OpenAI y Anthropic) importadas.


In [4]:
# Cargar nuestro GeoDataFrame enriquecido
RUTA_DATOS_PROCESADOS = os.path.join('..', '1_datos', '02_procesados')
RUTA_GPKG_ENRIQUECIDO = os.path.join(RUTA_DATOS_PROCESADOS, 'gdf_maestro_enriquecido.gpkg')
gdf = gpd.read_file(RUTA_GPKG_ENRIQUECIDO)

# Para el análisis Text-to-SQL, la columna de geometría no es útil y puede dar problemas.
# La quitamos para crear una tabla puramente de datos.
df_analisis = gdf.drop(columns=['geometry', 'GEOMETRY1_']) # Quitamos ambas por si acaso

# Crear una base de datos SQLite en memoria
engine = create_engine('sqlite:///:memory:')

# Cargar nuestros datos en una tabla llamada 'secciones' dentro de esa base de datos
df_analisis.to_sql('secciones', engine, index=False)

# Crear el objeto de base de datos que LangChain usará
db = SQLDatabase(engine=engine)

print("Base de datos en memoria creada y cargada con 65 secciones de Manzanillo.")
print("Tabla 'secciones' lista para ser consultada.")

Base de datos en memoria creada y cargada con 65 secciones de Manzanillo.
Tabla 'secciones' lista para ser consultada.


In [5]:
import os
from getpass import getpass

# Revisa si la variable de entorno para OpenAI no existe
if "OPENAI_API_KEY" not in os.environ:
  # Si no existe, pide al usuario que la ingrese de forma segura
  os.environ["OPENAI_API_KEY"] = getpass("Ingresa tu API Key de OpenAI: ")

# Hacemos lo mismo para la llave de Anthropic
if "ANTHROPIC_API_KEY" not in os.environ:
  os.environ["ANTHROPIC_API_KEY"] = getpass("Ingresa tu API Key de Anthropic: ")

print("API Keys configuradas para esta sesión.")

API Keys configuradas para esta sesión.


In [12]:
# --- Celda 3 Modificada ---

# 1. Inicializamos el LLM de OpenAI (GPT-4o) - SIN CAMBIOS
llm_openai = ChatOpenAI(model="gpt-4o", temperature=0)

# 2. Inicializamos el LLM de Anthropic - ¡AQUÍ ESTÁ EL CAMBIO!
# Cambiamos al identificador del modelo más reciente: Claude 3.5 Sonnet
llm_anthropic = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)
# Antiguo nombre comentado: model="claude-3-sonnet-20240229"


# El objeto de la base de datos es el mismo para ambos - SIN CAMBIOS
db = SQLDatabase(engine=engine)

# 3. Creamos el AGENTE para OpenAI - SIN CAMBIOS
agente_openai = create_sql_agent(
    llm=llm_openai,
    db=db,
    agent_type="openai-tools",
    verbose=True,
    prompt_suffix="Recuerda siempre responder en español."
)
print("✅ Agente de OpenAI (GPT-4o) creado.")

# 4. Creamos el AGENTE para Anthropic - SIN CAMBIOS EN LA LÓGICA
agente_anthropic = create_sql_agent(
    llm=llm_anthropic,
    db=db,
    verbose=True,
    prompt_suffix="Recuerda siempre responder en español."
)
print("✅ Agente de Anthropic (Claude 3.5 Sonnet) creado con método universal.")

✅ Agente de OpenAI (GPT-4o) creado.
✅ Agente de Anthropic (Claude 3.5 Sonnet) creado con método universal.


In [13]:
# --- Nueva Celda 5: Comparación de Modelos ---

# Definimos una pregunta estratégica interesante
pregunta_comparacion = "Quiero encontrar las 3 secciones con la mayor participación electoral. Para esto, busca las secciones con la tasa de participación promedio más alta"

print("==============================================")
print("🧠 Ejecutando con OpenAI (GPT-4o)...")
print("==============================================")

try:
    respuesta_openai = agente_openai.invoke(pregunta_comparacion)
    print("\n--- Respuesta de OpenAI ---")
    print(respuesta_openai['output'])
except Exception as e:
    print(f"Ocurrió un error con el agente de OpenAI: {e}")


print("\n\n==============================================")
print("✨ Ejecutando con Anthropic (Claude 3 Sonnet)...")
print("==============================================")

try:
    respuesta_anthropic = agente_anthropic.invoke(pregunta_comparacion)
    print("\n--- Respuesta de Anthropic ---")
    print(respuesta_anthropic['output'])
except Exception as e:
    print(f"Ocurrió un error con el agente de Anthropic: {e}")

🧠 Ejecutando con OpenAI (GPT-4o)...


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


secciones
Invoking: `sql_db_schema` with `{'table_names': 'secciones'}`



CREATE TABLE secciones (
	"ID" FLOAT, 
	"ENTIDAD" BIGINT, 
	"DISTRITO_F" BIGINT, 
	"DISTRITO_L" BIGINT, 
	"MUNICIPIO" BIGINT, 
	seccion BIGINT, 
	"TIPO" BIGINT, 
	"CONTROL" FLOAT, 
	lista_nominal_promedio FLOAT, 
	votos_totales_acumulados BIGINT, 
	votos_morena_acumulados BIGINT, 
	votos_oposicion_acumulados BIGINT, 
	partido_dominante TEXT, 
	pct_voto_morena FLOAT, 
	tasa_participacion_promedio FLOAT, 
	"DISTRITO" BIGINT, 
	"POBTOT" BIGINT, 
	"POBFEM" BIGINT, 
	"POBMAS" BIGINT, 
	"P_0A2" BIGINT, 
	"P_0A2_F" BIGINT, 
	"P_0A2_M" BIGINT, 
	"P_0A17" BIGINT, 
	"P_3YMAS" BIGINT, 
	"P_3YMAS_F" BIGINT, 
	"P_3YMAS_M" BIGINT, 
	"P_5YMAS" BIGINT, 
	"P_5YMAS_F" BIGINT, 
	"P_5YMAS_M" BIGINT, 
	"P_12YMAS" BIGINT, 
	"P_12YMAS_F" BIGINT, 
	"P_12YMAS_M" BIGINT, 
	"P_15YMAS" BIGINT, 
	"P_15YMAS_F" BIGINT, 

In [17]:
import pandas as pd
import geopandas as gpd
import os
from sqlalchemy import create_engine, text
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic 
from langchain.agents import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import logging

# Configurar logging para mejor debugging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AnalistaElectoralSQL:
    """
    Clase para manejar agentes SQL especializados en análisis electoral
    """
    
    def __init__(self, ruta_datos_procesados='../1_datos/02_procesados'):
        self.ruta_datos = ruta_datos_procesados
        self.engine = None
        self.db = None
        self.gdf = None
        self.agentes = {}
        
    def cargar_datos(self, archivo_gpkg='gdf_maestro_enriquecido.gpkg'):
        """Carga los datos geoespaciales y prepara la base de datos SQL"""
        try:
            ruta_completa = os.path.join(self.ruta_datos, archivo_gpkg)
            self.gdf = gpd.read_file(ruta_completa)
            
            # Preparar DataFrame para análisis (sin geometría)
            df_analisis = self.gdf.drop(columns=['geometry', 'GEOMETRY1_'], errors='ignore')
            
            # Crear base de datos en memoria
            self.engine = create_engine('sqlite:///:memory:')
            df_analisis.to_sql('secciones', self.engine, index=False, if_exists='replace')
            
            # Crear objeto de base de datos para LangChain
            self.db = SQLDatabase(engine=self.engine)
            
            # Verificar estructura de datos
            self._verificar_estructura_datos()
            
            logger.info(f"✅ Datos cargados: {len(df_analisis)} secciones de Manzanillo")
            return True
            
        except Exception as e:
            logger.error(f"❌ Error cargando datos: {e}")
            return False
    
    def _verificar_estructura_datos(self):
        """Verifica y muestra la estructura de los datos cargados"""
        try:
            with self.engine.connect() as conn:
                # Obtener información de columnas
                result = conn.execute(text("PRAGMA table_info(secciones)"))
                columnas = [row[1] for row in result.fetchall()]
                
                # Obtener muestra de datos
                result = conn.execute(text("SELECT * FROM secciones LIMIT 3"))
                muestra = result.fetchall()
                
            logger.info(f"Columnas disponibles: {columnas}")
            logger.info(f"Muestra de datos: {muestra[0] if muestra else 'Sin datos'}")
            
        except Exception as e:
            logger.error(f"Error verificando estructura: {e}")
    
    def crear_prompt_especializado(self):
        """Crea un prompt especializado para análisis electoral"""
        
        prompt_electoral = """
        Eres un analista electoral. Trabajas con la tabla 'secciones' que contiene datos de Manzanillo.
        
        INSTRUCCIONES:
        1. Primero usa PRAGMA table_info(secciones) para ver las columnas disponibles
        2. Luego ejecuta tu consulta SQL
        3. Responde en español con los resultados
        
        Mantén las consultas simples y directas.
        """
        
        return prompt_electoral
    
    def crear_agente_openai(self, modelo="gpt-4o", temperatura=0):
        """Crea agente especializado con OpenAI"""
        try:
            llm = ChatOpenAI(model=modelo, temperature=temperatura)
            
            agente = create_sql_agent(
                llm=llm,
                db=self.db,
                agent_type="openai-tools",
                verbose=True,
                prompt_suffix=self.crear_prompt_especializado(),
                max_iterations=5
            )
            
            self.agentes['openai'] = agente
            logger.info("✅ Agente OpenAI creado exitosamente")
            return agente
            
        except Exception as e:
            logger.error(f"❌ Error creando agente OpenAI: {e}")
            return None
    
    def crear_agente_anthropic(self, modelo="claude-3-5-sonnet-20241022", temperatura=0):
        """Crea agente especializado con Anthropic"""
        try:
            llm = ChatAnthropic(model=modelo, temperature=temperatura)
            
            agente = create_sql_agent(
                llm=llm,
                db=self.db,
                verbose=True,
                prompt_suffix=self.crear_prompt_especializado(),
                max_iterations=5
            )
            
            self.agentes['anthropic'] = agente
            logger.info("✅ Agente Anthropic creado exitosamente")
            return agente
            
        except Exception as e:
            logger.error(f"❌ Error creando agente Anthropic: {e}")
            return None
    
    def consultar_agente(self, pregunta, proveedor='openai'):
        """Ejecuta consulta con el agente especificado"""
        try:
            if proveedor not in self.agentes:
                raise ValueError(f"Agente {proveedor} no disponible")
            
            agente = self.agentes[proveedor]
            logger.info(f"🤖 Ejecutando consulta con {proveedor.upper()}")
            
            respuesta = agente.invoke(pregunta)
            
            resultado = {
                'proveedor': proveedor,
                'pregunta': pregunta,
                'respuesta': respuesta.get('output', respuesta),
                'exito': True
            }
            
            return resultado
            
        except Exception as e:
            logger.error(f"❌ Error en consulta con {proveedor}: {e}")
            return {
                'proveedor': proveedor,
                'pregunta': pregunta,
                'respuesta': f"Error: {str(e)}",
                'exito': False
            }
    
    def consultar_ambos_agentes(self, pregunta):
        """Ejecuta la misma consulta con ambos agentes para comparar"""
        resultados = {}
        
        for proveedor in ['openai', 'anthropic']:
            if proveedor in self.agentes:
                resultados[proveedor] = self.consultar_agente(pregunta, proveedor)
        
        return resultados
    
    def obtener_esquema_tabla(self):
        """Obtiene el esquema completo de la tabla para referencia"""
        try:
            with self.engine.connect() as conn:
                # Información de columnas
                result = conn.execute(text("PRAGMA table_info(secciones)"))
                esquema = []
                for row in result.fetchall():
                    esquema.append({
                        'columna': row[1],
                        'tipo': row[2],
                        'no_nulo': bool(row[3]),
                        'pk': bool(row[5])
                    })
                
                # Estadísticas básicas
                result = conn.execute(text("SELECT COUNT(*) as total_secciones FROM secciones"))
                total = result.fetchone()[0]
                
            return {
                'esquema': esquema,
                'total_secciones': total,
                'columnas': [col['columna'] for col in esquema]
            }
            
        except Exception as e:
            logger.error(f"Error obteniendo esquema: {e}")
            return None
    
    def queries_predefinidas(self):
        """Retorna un diccionario de queries comunes para el chatbot"""
        return {
            "participacion_alta": "¿Cuáles son las 5 secciones con mayor participación electoral promedio?",
            "participacion_baja": "¿Cuáles son las 5 secciones con menor participación electoral?",
            "perfil_demografico": "¿Cuál es el perfil demográfico promedio de las secciones con alta participación?",
            "correlacion_educacion": "¿Existe correlación entre nivel educativo y participación electoral?",
            "distribucion_edad": "¿Cómo se distribuye la población por grupos de edad en las secciones?",
            "resumen_general": "Dame un resumen general de las características electorales de Manzanillo"
        }

# Función de utilidad para inicialización rápida
def inicializar_analista_electoral(ruta_datos='../1_datos/02_procesados'):
    """
    Función de conveniencia para inicializar rápidamente el sistema completo
    """
    analista = AnalistaElectoralSQL(ruta_datos)
    
    # Cargar datos
    if not analista.cargar_datos():
        logger.error("❌ No se pudieron cargar los datos")
        return None
    
    # Crear ambos agentes
    agente_openai = analista.crear_agente_openai()
    agente_anthropic = analista.crear_agente_anthropic()
    
    if agente_openai is None and agente_anthropic is None:
        logger.error("❌ No se pudo crear ningún agente")
        return None
    
    logger.info("🎯 Sistema de análisis electoral inicializado exitosamente")
    return analista

# Función para testing rápido
def test_agentes_basico(analista):
    """Ejecuta un test básico de ambos agentes"""
    
    # Primero veamos qué columnas tenemos
    print("📋 COLUMNAS DISPONIBLES:")
    esquema = analista.obtener_esquema_tabla()
    for col in esquema['columnas']:
        print(f"  - {col}")
    
    # Pregunta muy simple para empezar
    pregunta_simple = "¿Cuántas secciones hay en total?"
    
    print(f"\n🧪 PREGUNTA SIMPLE: {pregunta_simple}")
    print("=" * 50)
    
    resultados = analista.consultar_ambos_agentes(pregunta_simple)
    
    for proveedor, resultado in resultados.items():
        print(f"\n🤖 {proveedor.upper()}:")
        print("-" * 30)
        if resultado['exito']:
            print(resultado['respuesta'])
        else:
            print(f"❌ Error: {resultado['respuesta']}")
    
    return resultados

# EJEMPLO DE USO:
if __name__ == "__main__":
    # Inicializar sistema
    analista = inicializar_analista_electoral()
    
    if analista:
        # Mostrar esquema de datos
        esquema = analista.obtener_esquema_tabla()
        print(f"\n📊 ESQUEMA DE DATOS:")
        print(f"Total secciones: {esquema['total_secciones']}")
        print(f"Columnas disponibles: {len(esquema['columnas'])}")
        
        # Ejecutar test básico
        test_agentes_basico(analista)
        
        # Mostrar queries predefinidas
        print(f"\n💡 QUERIES PREDEFINIDAS DISPONIBLES:")
        for clave, query in analista.queries_predefinidas().items():
            print(f"- {clave}: {query}")

INFO:__main__:Columnas disponibles: ['ID', 'ENTIDAD', 'DISTRITO_F', 'DISTRITO_L', 'MUNICIPIO', 'seccion', 'TIPO', 'CONTROL', 'lista_nominal_promedio', 'votos_totales_acumulados', 'votos_morena_acumulados', 'votos_oposicion_acumulados', 'partido_dominante', 'pct_voto_morena', 'tasa_participacion_promedio', 'DISTRITO', 'POBTOT', 'POBFEM', 'POBMAS', 'P_0A2', 'P_0A2_F', 'P_0A2_M', 'P_0A17', 'P_3YMAS', 'P_3YMAS_F', 'P_3YMAS_M', 'P_5YMAS', 'P_5YMAS_F', 'P_5YMAS_M', 'P_12YMAS', 'P_12YMAS_F', 'P_12YMAS_M', 'P_15YMAS', 'P_15YMAS_F', 'P_15YMAS_M', 'P_18YMAS', 'P_18YMAS_F', 'P_18YMAS_M', 'P_3A5', 'P_3A5_F', 'P_3A5_M', 'P_6A11', 'P_6A11_F', 'P_6A11_M', 'P_8A14', 'P_8A14_F', 'P_8A14_M', 'P_12A14', 'P_12A14_F', 'P_12A14_M', 'P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F', 'P_18A24_M', 'P_15A49_F', 'P_60YMAS', 'P_60YMAS_F', 'P_60YMAS_M', 'REL_H_M', 'POB0_14', 'POB15_64', 'POB65_MAS', 'POB_EDADNE', 'PROM_HNV', 'PNACENT', 'PNACENT_F', 'PNACENT_M', 'PNACOE', 'PNACOE_F', 'PNACOE_M', 'PRES2015


📊 ESQUEMA DE DATOS:
Total secciones: 65
Columnas disponibles: 253
📋 COLUMNAS DISPONIBLES:
  - ID
  - ENTIDAD
  - DISTRITO_F
  - DISTRITO_L
  - MUNICIPIO
  - seccion
  - TIPO
  - CONTROL
  - lista_nominal_promedio
  - votos_totales_acumulados
  - votos_morena_acumulados
  - votos_oposicion_acumulados
  - partido_dominante
  - pct_voto_morena
  - tasa_participacion_promedio
  - DISTRITO
  - POBTOT
  - POBFEM
  - POBMAS
  - P_0A2
  - P_0A2_F
  - P_0A2_M
  - P_0A17
  - P_3YMAS
  - P_3YMAS_F
  - P_3YMAS_M
  - P_5YMAS
  - P_5YMAS_F
  - P_5YMAS_M
  - P_12YMAS
  - P_12YMAS_F
  - P_12YMAS_M
  - P_15YMAS
  - P_15YMAS_F
  - P_15YMAS_M
  - P_18YMAS
  - P_18YMAS_F
  - P_18YMAS_M
  - P_3A5
  - P_3A5_F
  - P_3A5_M
  - P_6A11
  - P_6A11_F
  - P_6A11_M
  - P_8A14
  - P_8A14_F
  - P_8A14_M
  - P_12A14
  - P_12A14_F
  - P_12A14_M
  - P_15A17
  - P_15A17_F
  - P_15A17_M
  - P_18A24
  - P_18A24_F
  - P_18A24_M
  - P_15A49_F
  - P_60YMAS
  - P_60YMAS_F
  - P_60YMAS_M
  - REL_H_M
  - POB0_14
  - POB15_64
  

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_db_list_tables` with `{}`


secciones

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

SELECT COUNT(*) as total_secciones FROM secciones;

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


La consulta parece correcta. Vamos a ejecutarla.

Action: sql_db_query
Action Input: SELECT COUNT(*) as total_secciones FROM secciones;
[(65,)]

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


La consulta me ha devuelto el número total de secciones.

Final Answer: Hay 65 secciones en total.

> Finished chain.

🤖 OPENAI:
------------------------------
Hay un total de 65 secciones en la base de datos.

🤖 ANTHROPIC:
------------------------------
Hay 65 secciones en total.

💡 QUERIES PREDEFINIDAS DISPONIBLES:
- participacion_alta: ¿Cuáles son las 5 secciones con mayor participación electoral promedio?
- participacion_baja: ¿Cuáles son las 5 secciones con menor participación electoral?
- perfil_demografico: ¿Cuál es el perfil demográfico promedio de las secciones con alta participación?
- correlacion_educacion: ¿Existe correlación entre nivel educativo y participación electoral?
- distribucion_edad: ¿Cómo se distribuye la población por grupos de edad en las secciones?
- resumen_general: Dame un resumen general de las características electorales de Manzanillo


In [18]:
# Luego en otra celda:
analista = inicializar_analista_electoral()

if analista:
    test_agentes_basico(analista)

INFO:__main__:Columnas disponibles: ['ID', 'ENTIDAD', 'DISTRITO_F', 'DISTRITO_L', 'MUNICIPIO', 'seccion', 'TIPO', 'CONTROL', 'lista_nominal_promedio', 'votos_totales_acumulados', 'votos_morena_acumulados', 'votos_oposicion_acumulados', 'partido_dominante', 'pct_voto_morena', 'tasa_participacion_promedio', 'DISTRITO', 'POBTOT', 'POBFEM', 'POBMAS', 'P_0A2', 'P_0A2_F', 'P_0A2_M', 'P_0A17', 'P_3YMAS', 'P_3YMAS_F', 'P_3YMAS_M', 'P_5YMAS', 'P_5YMAS_F', 'P_5YMAS_M', 'P_12YMAS', 'P_12YMAS_F', 'P_12YMAS_M', 'P_15YMAS', 'P_15YMAS_F', 'P_15YMAS_M', 'P_18YMAS', 'P_18YMAS_F', 'P_18YMAS_M', 'P_3A5', 'P_3A5_F', 'P_3A5_M', 'P_6A11', 'P_6A11_F', 'P_6A11_M', 'P_8A14', 'P_8A14_F', 'P_8A14_M', 'P_12A14', 'P_12A14_F', 'P_12A14_M', 'P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F', 'P_18A24_M', 'P_15A49_F', 'P_60YMAS', 'P_60YMAS_F', 'P_60YMAS_M', 'REL_H_M', 'POB0_14', 'POB15_64', 'POB65_MAS', 'POB_EDADNE', 'PROM_HNV', 'PNACENT', 'PNACENT_F', 'PNACENT_M', 'PNACOE', 'PNACOE_F', 'PNACOE_M', 'PRES2015

📋 COLUMNAS DISPONIBLES:
  - ID
  - ENTIDAD
  - DISTRITO_F
  - DISTRITO_L
  - MUNICIPIO
  - seccion
  - TIPO
  - CONTROL
  - lista_nominal_promedio
  - votos_totales_acumulados
  - votos_morena_acumulados
  - votos_oposicion_acumulados
  - partido_dominante
  - pct_voto_morena
  - tasa_participacion_promedio
  - DISTRITO
  - POBTOT
  - POBFEM
  - POBMAS
  - P_0A2
  - P_0A2_F
  - P_0A2_M
  - P_0A17
  - P_3YMAS
  - P_3YMAS_F
  - P_3YMAS_M
  - P_5YMAS
  - P_5YMAS_F
  - P_5YMAS_M
  - P_12YMAS
  - P_12YMAS_F
  - P_12YMAS_M
  - P_15YMAS
  - P_15YMAS_F
  - P_15YMAS_M
  - P_18YMAS
  - P_18YMAS_F
  - P_18YMAS_M
  - P_3A5
  - P_3A5_F
  - P_3A5_M
  - P_6A11
  - P_6A11_F
  - P_6A11_M
  - P_8A14
  - P_8A14_F
  - P_8A14_M
  - P_12A14
  - P_12A14_F
  - P_12A14_M
  - P_15A17
  - P_15A17_F
  - P_15A17_M
  - P_18A24
  - P_18A24_F
  - P_18A24_M
  - P_15A49_F
  - P_60YMAS
  - P_60YMAS_F
  - P_60YMAS_M
  - REL_H_M
  - POB0_14
  - POB15_64
  - POB65_MAS
  - POB_EDADNE
  - PROM_HNV
  - PNACENT
  - PNACENT_F
 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_db_list_tables` with `{}`


secciones

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

SELECT COUNT(*) as total_secciones FROM secciones;

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


La consulta parece correcta. Vamos a ejecutarla.

Action: sql_db_query
Action Input: SELECT COUNT(*) as total_secciones FROM secciones;
[(65,)]

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


La consulta me muestra que hay 65 secciones en total.

Final Answer: Hay 65 secciones en total en la base de datos.

> Finished chain.

🤖 OPENAI:
------------------------------
Hay un total de 65 secciones en la base de datos.

🤖 ANTHROPIC:
------------------------------
Hay 65 secciones en total en la base de datos.


In [19]:
# O comparar ambos agentes
pregunta = "¿Cuáles son las 5 secciones con más adultos mayores"
resultados = analista.consultar_ambos_agentes(pregunta)

for proveedor, resultado in resultados.items():
    print(f"\n🤖 {proveedor.upper()}:")
    print(resultado['respuesta'])

INFO:__main__:🤖 Ejecutando consulta con OPENAI




> Entering new SQL Agent Executor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_db_list_tables` with `{}`


secciones

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

SELECT seccion, porc_adultos_mayores, POBTOT
FROM secciones
WHERE porc_adultos_mayores IS NOT NULL
ORDER BY porc_adultos_mayores DESC
LIMIT 5;

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


The query looks correct. Let's execute it to get the results.

Action: sql_db_query
Action Input: SELECT seccion, porc_adultos_mayores, POBTOT
FROM secciones
WHERE porc_adultos_mayores IS NOT NULL
ORDER BY porc_adultos_mayores DESC
LIMIT 5;
[(201, 42.857142857142854, 7), (224, 22.697368421052634, 304), (220, 22.201492537313435, 536), (212, 18.181818181818183, 572), (222, 18.11320754716981, 530)]

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


I can now provide the final answer with the 5 sections that have the highest percentage of elderly people.

Final Answer: Las 5 secciones con mayor porcentaje de adultos mayores son:

1. Sección 201: 42.86% de adultos mayores (población total: 7 personas)
2. Sección 224: 22.70% de adultos mayores (población total: 304 personas)
3. Sección 220: 22.20% de adultos mayores (población total: 536 personas)
4. Sección 212: 18.18% de adultos mayores (población total: 572 personas)
5. Sección 222: 18.11% de adultos mayores (población total: 530 personas)

Nota: Es importante mencionar que la sección 201, aunque tiene el porcentaje más alto, tiene una población total muy pequeña (solo 7 personas), lo cual podría no ser muy representativo en comparación con las otras secciones que tienen poblaciones mucho más grandes.

> Finished chain.

🤖 OPENAI:
Las 5 secciones con más adultos mayores son las siguientes, junto con el número de adultos mayores en cada una:

1. Sección 251: 655 adultos mayores
2.

In [20]:
# Preguntas una por una con un solo agente
pregunta = "¿Cuáles son las 3 secciones con mayor porcentaje de voto por morena?"
resultado = analista.consultar_agente(pregunta, 'openai')
print(resultado['respuesta'])

INFO:__main__:🤖 Ejecutando consulta con OPENAI




> Entering new SQL Agent Executor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_db_list_tables` with `{}`


secciones

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

```sql
SELECT seccion, pct_voto_morena FROM secciones ORDER BY pct_voto_morena DESC LIMIT 3;
```

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_db_query` with `{'query': 'SELECT seccion, pct_voto_morena FROM secciones ORDER BY pct_voto_morena DESC LIMIT 3;'}`


[(229, 41.62493043962159), (247, 40.77770130763937), (235, 39.905660377358494)]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Las 3 secciones con mayor porcentaje de voto por Morena son:

1. Sección 229 con un 41.62% de votos.
2. Sección 247 con un 40.78% de votos.
3. Sección 235 con un 39.91% de votos.

> Finished chain.
Las 3 secciones con mayor porcentaje de voto por Morena son:

1. Sección 229 con un 41.62% de votos.
2. Sección 247 con un 40.78% de votos.
3. Sección 235 con un 39.91% de votos.
